<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/Kaggle_Toxic_Comments_Feb23/Model-6-keras_lstmConv_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print("Name of GPU : {}".format(torch.cuda.get_device_name(0)))
print("# of GPU : {}".format(torch.cuda.device_count()))


Name of GPU : Tesla K80
# of GPU : 1


In [0]:
import pandas as pd
import string
import time
import random
import pandas as pd
import numpy as np
import gc
import re
import torch
from torchtext import data
import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm
import zipfile
import gzip
import keras.backend as K

tqdm.pandas(desc='Progress')
from collections import Counter
from textblob import TextBlob
from nltk import word_tokenize
import torch
import torch.nn as nn
import random
import keras
import numpy as np
from keras.callbacks import Callback
from keras.optimizers import Optimizer
from keras import backend as K, initializers, regularizers, constraints
from keras.regularizers import l2
from keras.engine.topology import Layer

from keras.layers import K, Activation
from keras.engine import Layer
from keras.layers import PReLU, Conv1D, Conv2D, Input, SpatialDropout1D, Embedding, Dense, Activation, Bidirectional, LSTM,CuDNNLSTM, CuDNNGRU, GlobalMaxPooling1D,GlobalAveragePooling1D, concatenate, Dropout
from keras import optimizers
from keras.optimizers import Adam, RMSprop, Nadam
from keras.models import Model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk

# cross validation and metrics
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import model_selection,linear_model,metrics
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, roc_auc_score
from torch.optim.optimizer import Optimizer
from unidecode import unidecode

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/kaggle_toxic_comments"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
best_model.hdf5			   test.csv
ext_fasttextwiki_embeddings.npy    test_features.npy
ext_glove_embeddings.npy	   test_labels.csv
ext_lexvec_embeddings.npy	   toxic_lookup_en_wiki_fasttext_300d.npy
features.npy			   train.csv
inputed_lexvec_embeddings.npy	   word_index.npy
local_fasttextwiki_embeddings.npy  x_test.npy
Model-runs			   x_train.npy
sample_submission.csv		   y_train.npy
submission_baseline_lr.csv


In [0]:
#configs
embed_size = 300 # how big is each word vector
max_features = 120000 # how many unique words to use (i.e num rows in embedding vector) ##sum([word[1] for word in vocab[:120000]])=92%##
maxlen = 250 # max number of words in a question to use ## train.num_words.quantile(q=0.96, interpolation='nearest')=258 ##
batch_size = 512 # how many samples to process at once
n_epochs = 5 # how many times to iterate over all samples
n_splits = 5 # Number of K-fold Splits

SEED = 2019
def seed_everything(seed=2019):
    random.seed(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()


In [0]:
os.chdir("/content/drive/My Drive/kaggle_toxic_comments/")
x_train = np.load("x_train.npy")
x_test = np.load("x_test.npy")
y_train = np.load("y_train.npy")
features = np.load("features.npy")
test_features = np.load("test_features.npy")
word_index = np.load("word_index.npy").item()
ext_lexvec_embeddings = np.load("ext_lexvec_embeddings.npy")
ext_glove_embeddings = np.load("ext_glove_embeddings.npy")
ext_fasttextwiki_embeddings = np.load("ext_fasttextwiki_embeddings.npy")
imputed_lexvec_embeddings = np.load("/content/drive/My Drive/kaggle_toxic_comments/inputed_lexvec_embeddings.npy")
#local_fasttextwiki_embeddings = np.load("local_fasttextwiki_embeddings.npy")

In [0]:
# Train simple LSTM using different embeddings
# Top compeitions- QIQC and Toxic comments
# ensembles (stacking and blending)
# Pretrained models- ULMFit and BERT
# Gensim and fasttext text classification
# Other SOTAs

In [0]:
##Model-6: https://github.com/mattmotoki/toxic-comment-classification/blob/master/code/modeling/Refine%20RNN%20Version%201.ipynb
#Prelu
#Nadam
#Attention with Sequence
#Multiple seeds emsemble
#multiple embeddings ensemble

In [0]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

            


class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=4590., mode='triangular',gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}
        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):     
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v) 
        K.set_value(self.model.optimizer.lr, self.clr())
        
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

# A Capsule Implement with Pure Keras
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [0]:
def get_model(embed_name, n_recurrent, n_filters, l2_penalty=0.0001):
    K.clear_session()
    embed_matrix = np.load(embed_name)
    inp = Input(shape=(maxlen,))
    emb   = Embedding(max_features, embed_size, weights=[embed_matrix], trainable=False)(inp)
    x   = SpatialDropout1D(rate=0.2)(emb)
    x   = Bidirectional(CuDNNGRU(n_recurrent, return_sequences=True, kernel_regularizer=l2(l2_penalty),recurrent_regularizer=l2(l2_penalty)))(x)
    x   = Conv1D(64, kernel_size=1, padding='valid', kernel_initializer='glorot_normal')(x)
    x = PReLU()(x)
    x_att = AttentionWithContext()(x)
    x_max   = GlobalMaxPooling1D()(x)
    x_avg   = GlobalAveragePooling1D()(x)
    
    x1   = SpatialDropout1D(rate=0.2)(emb)
    x1   = Bidirectional(CuDNNGRU(n_recurrent, return_sequences=True, kernel_regularizer=l2(l2_penalty),recurrent_regularizer=l2(l2_penalty)))(x1)
    x1   = Conv1D(2*64, kernel_size=1, padding='valid', kernel_initializer='glorot_normal')(x1)
    x1 = PReLU()(x1)
    x1_att = AttentionWithContext()(x1)
    x1_max   = GlobalMaxPooling1D()(x1)
    x1_avg   = GlobalAveragePooling1D()(x1)
    
    c   = concatenate([x_max,x_avg,x_att,x1_max,x1_avg,x1_att])
    out = Dense(6, activation='sigmoid')(c)
    model=Model(inputs=inp,outputs=out)
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=["accuracy"])
    return model


def pred_model(emb, n_splits, seed):
  
    kfold = KFold(n_splits=n_splits, random_state=seed, shuffle=True)
    y_test = np.zeros((x_test.shape[0], 6))
    for i, (train_index, valid_index) in enumerate(kfold.split(x_train)):
        X_train, X_val, Y_train, Y_val = x_train[train_index], x_train[valid_index], y_train[train_index], y_train[valid_index]
        file_path = "best_model.hdf5"
        check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,save_best_only = True, mode = "min")
        ra_val = RocAucEvaluation(validation_data=(X_val, Y_val), interval = 1)
        early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)
        model = get_model(n_recurrent=50, n_filters=50, embed_name=emb, l2_penalty=0.0001)
        print("### Model: {} with seed: {}  for fold no. {} ####".format(emb, seed, i))
        clr = CyclicLR(base_lr=1e-6, max_lr=1e-3,step_size=2524., mode='exp_range',gamma=0.99994)
        model.fit(X_train, Y_train, batch_size = 128, epochs = 5, validation_data = (X_val, Y_val), verbose = 1, callbacks = [clr, ra_val, check_point, early_stop])
        y_pred = model.predict([X_val], batch_size=batch_size, verbose=2)
        y_test += model.predict([x_test], batch_size=1024, verbose=1)/5
        os.remove(file_path)
    return y_test
  

In [0]:
for emb in ['ext_lexvec_embeddings.npy', 'ext_fasttext_wiki_embeddings.npy', 'ext_glove_embeddings.npy', 'imputed_lexvec_embeddings.npy']:    
    for seed in [0, 1, 2]:
        pred = pred_model(emb, n_splits=10, seed=seed)
        pred += pred/12

### Model: ext_lexvec_embeddings.npy with seed: 0  for fold no. 0 ####
Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 246s 2ms/step - loss: 0.2379 - acc: 0.9457 - val_loss: 0.0917 - val_acc: 0.9800

 ROC-AUC - epoch: 1 - score: 0.939959

Epoch 00001: val_loss improved from inf to 0.09166, saving model to best_model.hdf5
Epoch 2/5
143613/143613 [==============================] - 244s 2ms/step - loss: 0.0748 - acc: 0.9803 - val_loss: 0.0621 - val_acc: 0.9821

 ROC-AUC - epoch: 2 - score: 0.969875

Epoch 00002: val_loss improved from 0.09166 to 0.06206, saving model to best_model.hdf5
Epoch 3/5
143613/143613 [==============================] - 244s 2ms/step - loss: 0.0584 - acc: 0.9817 - val_loss: 0.0523 - val_acc: 0.9830

 ROC-AUC - epoch: 3 - score: 0.977145

Epoch 00003: val_loss improved from 0.06206 to 0.05230, saving model to best_model.hdf5
Epoch 4/5
143613/143613 [==============================] - 245s 2ms/step - loss: 

In [0]:
submission = pd.read_csv("./sample_submission.csv")
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = (pred)
submission.to_csv("submission_model6_Feb24_auc_988317.csv", index = False)

In [0]:

y_test = np.zeros((x_test.shape[0], 6))

In [56]:
y_test.shape

(153164, 6)